In [1]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
# lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

/home/ados/anaconda3/envs/fastchat_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
new_dataset[2]

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})

In [175]:
result = collection.query(query_texts="이 과제는 벵골어로 된 문장을 영어로 번역하는 것입니다.\nQ: অথবা আপনি হয়ত বলবেন, 'না, আমার খেয়াল হয়েছে যে আমি আসলেই মহিলাদ", n_results=min(n_results, len(query_ids)), include=['distances', 'documents'])
search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [ ]:
distances, documents

In [ ]:
dataset_ = dedup_non_pair(dataset, 'dpo')

In [26]:
from fastchat.train.data_modules.dpo_dataset import ados_DPODataset

dpo_dataset = ados_DPODataset(
    dataset_path="/data/llm_datasets/custom/ados/dpo/ados_dpo_v2.json",
    data_format='qwen',
)
dpo_datamodule = dpo_dataset.make_dpo_data_module()

In [2]:
def load_raw_dataset(dataset_path, split='train'):
    if dataset_path.endswith("json") or dataset_path.endswith("json.kr"):
        try:
            dataset = load_dataset("json", data_files=dataset_path, split=split)
        except:
            print("Not able to read records in the JSON file, read from json reader...")
            dataset = load_json_dataset(dataset_path)
    elif dataset_path.endswith("parquet"):
        dataset = load_dataset("parquet", data_files=dataset_path, split=split)
    else:
        dataset = load_dataset(dataset_path, split=split)
    return dataset

import json
def load_json_dataset(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)

    return data

import os
import json
from typing import Union, List
def dataset_to_json(dataset: Union[List[dict], Dataset], path: str):
    if isinstance(dataset, Dataset):
        dataset = [data for data in dataset]
    
    if os.path.dirname(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding='utf-8') as json_file:
        json.dump(dataset, json_file, indent=4, ensure_ascii=True)
        
    print(f"saved at {path}")

In [153]:
raw_dataset = load_raw_dataset("/data/llm_datasets/sftv5/processed/AkwardCorrection/mr-tydi-corpus_AkwardCorrectionData.json")
new_dataset = []
for idx, data in enumerate(raw_dataset):
    lists = data['conversation'][1]['value']
    answer = data['conversation'][3]['value']
    
    new_dataset.append({
        'id': f"mr_tydi_akward_correction_{idx}",
        'task': 'Akward Correction',
        'conversation': [
            {
                "from": "human",
                "value": f"주어진 문장 리스트에서 문맥 상 어울리지 않는 문장을 찾으세요.\n#문장 리스트: {lists}",
            },
            {
                "from": "gpt",
                "value":  f"주어진 문장 리스트에서 문맥상 어울리지 않는 문장은 다음과 같습니다.\n{answer}\n\n나머지 문장들은 모두 이와는 전혀 관련이 없는 문장입니다.",
            },
        ]
    })
dataset_to_json(new_dataset, "/data/llm_datasets/sftv5/processed/AkwardCorrection/refined/mr_tydi_akward_correction.json")

In [156]:
raw_dataset = load_raw_dataset("/data/llm_datasets/sftv5/processed/SummaryInference/TechnicalSciencesSummaryInferenceData.json")

In [186]:
import random
new_dataset = []
for idx, data in enumerate(raw_dataset):
    question = data['conversation'][0]['value']
    refer_lists = data['conversation'][2]['value']
    answer = data['conversation'][4]['value']
    
    template = random.choice([
        "당신에게 주어진 이 문장들은 어떤 논문의 요약으로 이로부터 논문의 본문을 작성해야합니다. 주어진 요약문장과 참조할 문장들을 토대로 논문을 작성하세요.\n#논문 요약: {summary}\n#참조할 문장들: {refer}",
        "이 문제에서는 어떤 논문의 요약본이 주어집니다. 참조 문장을 참고하여 요약본을 구체화하여 논문의 본문을 작성하세요.\n#요약본: {summary}\n#참조 문장: {refer}"
    ])
    
    new_dataset.append({
        'id': f"aihub_tech_sci_summary_infer_{idx}",
        'task': 'Summary Inference',
        'conversation': [
            {
                "from": "human",
                "value": template.format(summary=question, refer=refer_lists),
            },
            {
                "from": "gpt",
                "value":  answer,
            },
        ]
    })
dataset_to_json(new_dataset, "/data/llm_datasets/sftv5/processed/SummaryInference/refined/aihub_tech_sci_summary_infer.json")

saved at /data/llm_datasets/sftv5/processed/SummaryInference/refined/aihub_tech_sci_summary_infer.json


In [3]:
# long short제외 dedup
# paths = glob.glob('/data/llm_datasets/sftv5/processed/Translation/*.json')
paths = ['/data/llm_datasets/dpov3/1reformat/aihub_enko_society_dpo.json',
        '/data/llm_datasets/dpov3/1reformat/aihub_enko_tech_dpo.json']
for path in paths:
    # path = '/data/llm_datasets/sftv5/processed/Translation/aihub_expert_enko.json'
    dst = os.path.join("/data/llm_datasets/dpov3/4dedup/", os.path.basename(path))
    dataset = load_raw_dataset(path)
    if 'conversation' in dataset.features:
        dataset = dataset.rename_column('conversation', 'conversations')

    # apply dedups
    print("dedup_non_pair")
    dataset = dedup_non_pair(dataset, 'dpo')
    print("dedup_repetition")
    dataset = dedup_repetition(dataset, 'dpo')
    print("dedup_math")
    dataset = dedup_math(dataset, 'dpo')
    print("dedup_too_much_token")
    dataset = dedup_too_much_token(dataset, 'dpo', model_name='DIE-10_7B_sftv5_daily-100')
    print("dedup_short")
    dataset = dedup_short(dataset, 'dpo', 5)
    # do similarity dedup if needed
    # do code refine if needed

    dataset_to_json(dataset, dst)

dedup_non_pair


Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400878/400878 [00:02<00:00, 143726.31 examples/s]


0/400878 deduped
dedup_repetition


Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400878/400878 [00:35<00:00, 11274.63 examples/s]


364/400878 deduped
dedup_math


...

0/336960 deduped
dedup_too_much_token


Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336960/336960 [00:05<00:00, 57408.22 examples/s]


0/336960 deduped
dedup_short


Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336960/336960 [00:05<00:00, 57774.97 examples/s]


0/336960 deduped
saved at /data/llm_datasets/dpov3/4dedup/aihub_enko_tech_dpo.json


In [ ]:
import os
for path in sftv5_list:
    dst = os.path.join("/data/llm_datasets/sftv5/deduped/", os.path.basename(path))
    dataset = load_raw_dataset(dst)
    
    print(dst, dataset)

In [5]:
def uniform_select(dataset, division_factor):
    return dataset.select(list(range(0, len(dataset), division_factor)))

In [6]:
rebal_list = [
    "/data/llm_datasets/dpov3/4dedup/aihub_enko_tech_dpo.json",
    "/data/llm_datasets/dpov3/4dedup/aihub_enko_society_dpo.json",
]
rebal_factor = [60, 80]

for i in range(2):
    path = rebal_list[i]
    dst = path.replace("4dedup", "5rebal")
    dataset = load_raw_dataset(path)
    dataset = uniform_select(dataset, rebal_factor[i])
    dataset_to_json(dataset, dst)


saved at /data/llm_datasets/dpov3/5rebal/aihub_enko_tech_dpo.json


Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1855.89it/s]
Generating train split: 400514 examples [00:03, 125615.00 examples/s]


saved at /data/llm_datasets/dpov3/5rebal/aihub_enko_society_dpo.json


In [40]:
import random
"/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json", # daily
"/data/llm_datasets/custom/vicuna_format/sharegpt_gpt4.json", # daily
"/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json", # daily
"/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json", # daily

v4_list = [
    "/data/llm_datasets/sftv5/rebal/aihub_book_summary.json",
    "/data/llm_datasets/sftv5/rebal/aihub_law_summary.json",
    "/data/llm_datasets/sftv5/rebal/naver-news-summarization.json",
    "/data/llm_datasets/custom/vicuna_format/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/custom/refined/alpaca-gpt4-korean_dedup2.json", 
    "/data/llm_datasets/custom/vicuna_format/korquad-chat-vicuna.json", 
    "/data/llm_datasets/custom/refined/wizardlm_orca_vicuna_dedup2.json",
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(enko)-10000.json",
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json",
]

task_list = [
    '/data/llm_datasets/sftv5/deduped/wiki_qa_front_infer.json',
    '/data/llm_datasets/sftv5/deduped/wiki_qa_mid_key_infer.json',
    '/data/llm_datasets/sftv5/deduped/aihub_tech_sci_summary_infer.json',
    '/data/llm_datasets/sftv5/deduped/korea_spelling_correction.json',
    '/data/llm_datasets/sftv5/deduped/aihub_interface_spelling_correction.json',
    '/data/llm_datasets/sftv5/rebal/kocommercial_rear_infer.json',
    '/data/llm_datasets/sftv5/rebal/kocommercial_mid_word_infer.json',
    '/data/llm_datasets/sftv5/rebal/kocommercial_rearrange_sen.json',
    '/data/llm_datasets/sftv5/rebal/kocommercial_rearrange.json',
    '/data/llm_datasets/sftv5/rebal/mr_tydi_akward_correction.json',
    "/data/llm_datasets/sftv5/rebal/aihub_cnko_society.json",
    "/data/llm_datasets/sftv5/rebal/aihub_cnko_tech.json",
    "/data/llm_datasets/sftv5/rebal/aihub_enko_society.json",
    "/data/llm_datasets/sftv5/rebal/aihub_enko_tech.json",
    "/data/llm_datasets/sftv5/rebal/aihub_expert_enko.json",
    "/data/llm_datasets/sftv5/rebal/aihub_jpko.json",
]

# random.shuffle(task_list)


In [18]:
# 이미 학습된 데이터는 10분의 1만 사용
for i in range(len(v4_list)):
    path = v4_list[i]
    dst = os.path.join("/data/llm_datasets/sftv5/v4_repeat/", os.path.basename(path))
    print(dst)
    dataset = load_raw_dataset(path)
    dataset = uniform_select(dataset, 10)
    dataset_to_json(dataset, dst)


/data/llm_datasets/sftv5/v4_repeat/aihub_book_summary.json
saved at /data/llm_datasets/sftv5/v4_repeat/aihub_book_summary.json
/data/llm_datasets/sftv5/v4_repeat/aihub_law_summary.json
saved at /data/llm_datasets/sftv5/v4_repeat/aihub_law_summary.json
/data/llm_datasets/sftv5/v4_repeat/naver-news-summarization.json
saved at /data/llm_datasets/sftv5/v4_repeat/naver-news-summarization.json
/data/llm_datasets/sftv5/v4_repeat/koalpaca_v1.1-vicuna.json
saved at /data/llm_datasets/sftv5/v4_repeat/koalpaca_v1.1-vicuna.json
/data/llm_datasets/sftv5/v4_repeat/alpaca-gpt4-korean_dedup2.json
saved at /data/llm_datasets/sftv5/v4_repeat/alpaca-gpt4-korean_dedup2.json
/data/llm_datasets/sftv5/v4_repeat/korquad-chat-vicuna.json
saved at /data/llm_datasets/sftv5/v4_repeat/korquad-chat-vicuna.json
/data/llm_datasets/sftv5/v4_repeat/wizardlm_orca_vicuna_dedup2.json
saved at /data/llm_datasets/sftv5/v4_repeat/wizardlm_orca_vicuna_dedup2.json
/data/llm_datasets/sftv5/v4_repeat/sharegpt_V3_format_translati

In [41]:
v4_repeat_list = [
    "/data/llm_datasets/sftv5/v4_repeat/aihub_book_summary.json",
    "/data/llm_datasets/sftv5/v4_repeat/aihub_law_summary.json",
    "/data/llm_datasets/sftv5/v4_repeat/naver-news-summarization.json",
    "/data/llm_datasets/sftv5/v4_repeat/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/sftv5/v4_repeat/alpaca-gpt4-korean_dedup2.json",
    "/data/llm_datasets/sftv5/v4_repeat/korquad-chat-vicuna.json",
    "/data/llm_datasets/sftv5/v4_repeat/wizardlm_orca_vicuna_dedup2.json",
    # "/data/llm_datasets/sftv5/v4_repeat/sharegpt_V3_format_translation(enko)-10000.json",
    # "/data/llm_datasets/sftv5/v4_repeat/sharegpt_V3_format_translation(koen)-10000.json",
]

v5_task_list = task_list + v4_repeat_list

In [47]:
#sftv5_task_div30

sftv5_task_div30 = concatenate_datasets([load_sft_dataset(path, 'train') for path in v5_task_list])
sftv5_task_div30 = uniform_select(sftv5_task_div30, 30)
dataset_to_json(sftv5_task_div30, "/data/llm_datasets/sftv5/v5_task_repeat/sftv5_task_div30.json")

saved at /data/llm_datasets/sftv5/v5_task_repeat/sftv5_task_div30.json


In [62]:
v5_daily_list = [
    "/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json", # daily
    "/data/llm_datasets/custom/vicuna_format/sharegpt_gpt4.json", # daily
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json", # daily
    "/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json", # daily
    "/data/llm_datasets/sftv5/v5_task_repeat/sftv5_task_div30.json" # task_div
]
print(v5_daily_list)

['/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json', '/data/llm_datasets/custom/vicuna_format/sharegpt_gpt4.json', '/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json', '/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json', '/data/llm_datasets/sftv5/v5_task_repeat/sftv5_task_div30.json']


In [44]:
for path in task_list:
    dataset = load_raw_dataset(path)
    print(len(dataset), '\n', dataset[0])
    print()

68173 
 {'task': 'Front Inference', 'id': 'wiki_qa_front_infer_0', 'system': None, 'conversations': [{'from': 'human', 'value': "다음 텍스트에서 제공된 문맥을 정확히 이해하고, 앞에 올 문장을 자연스럽고 문맥에 맞게 작성하세요. 문장은 이후 내용과 논리적으로 연결되어야 합니다.\n#텍스트: 이들 식물은 사과와 같은 향이 나기 때문에 '캐모마일'이라고 불립니다. 캐모마일은 노란색 꽃을 가진 식물이며, 6월 중순에서 7월 하순까지 꽃을 피워요. 추위에 강한 편이며, 양지에서 잘 자라요. 꽃은 낮에는 피고 밤에는 오므라져 있으며, 일주일 정도 꽃이 핀다고 합니다. 씨받이는 매우 쉬우며, 싹도 잘 나기 때문에 직접 씨를 받을 수 있습니다. 캐모마일은 양지 바르고 배수가 좋은 사질토에서 잘 피워요. 수확한 꽃은 통풍이 잘 되는 응달에서 말려 저장해요. 캐모마일의 방향유는 긴장 완화, 두통 등의 통증 완화에 효과가 있으며, 몸을 따뜻하게 해주는 효과도 있습니다. 차로 마시면 위장장애의 완화에 도움을 줄 수 있습니다. 캐모마일은 카페인 성분이 없는 허브이며, 좀벌레를 없애는 데 효과적인 허브식물입니다."}, {'from': 'gpt', 'value': "캐모마일은 국화과에 속하는 식물로, 저먼캐모마일, 캐모마일, 카미레 등이 포함됩니다. 이들 식물은 사과와 같은 향이 나기 때문에 '캐모마일'이라고 불립니다. 캐모마일은 노란색 꽃을 가진 식물이며, 6월 중순에서 7월 하순까지 꽃을 피워요. 추위에 강한 편이며, 양지에서 잘 자라요. 꽃은 낮에는 피고 밤에는 오므라져 있으며, 일주일 정도 꽃이 핀다고 합니다. 씨받이는 매우 쉬우며, 싹도 잘 나기 때문에 직접 씨를 받을 수 있습니다. 캐모마일은 양지 바르고 배수가 좋은 사질토에서 잘 피워요. 수확한 꽃은 통풍이 잘 되는 응달에서 말려 저장해요. 캐모마일의 방향유는 긴장 완화, 두통 등의 통증 완화에 효과가 있으며, 몸을 따뜻하

In [61]:
for path in v5_daily_list:
    dataset = load_raw_dataset(path)
    print(path, len(dataset), '\n', dataset[0])
    print()

/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json 1316 
 {'id': 'gpt_evol_1.3k_0', 'conversations': [{'from': 'human', 'value': '다음 문장을 복수형에서 단수형으로 변환하세요.\n차들이 밖에 주차되어 있습니다.'}, {'from': 'gpt', 'value': '차가 밖에 주차되어 있습니다.'}]}

/data/llm_datasets/custom/vicuna_format/sharegpt_gpt4.json 6154 
 {'conversations': [{'from': 'human', 'value': '採用優雅現代中文，用中文繁體字型，回答以下問題。為所有標題或專用字詞提供對應的英語翻譯：Using scholarly style, summarize in detail James Barr\'s book "Semantics of Biblical Language". Provide examples supplied in the book and give corresponding page numbers wherever applicable.'}, {'from': 'gpt', 'value': '《聖經語言的語義學》（Semantics of Biblical Language）是詹姆斯·巴爾（James Barr）於1961年出版的一本研究聖經語言的專著。在這部作品中，巴爾對聖經希伯來語和希臘語的語言學特徵進行了深入的探討，他批判了當時盛行的聖經研究方法，主張應該將語言學原理應用於聖經文本的詮釋，以便更準確地理解聖經的意思。\n\n巴爾在《聖經語言的語義學》中提出了幾個重要的觀點。首先，他主張語言學和語言學家在聖經研究中的角色應該受到重視。他認為，應該避免將聖經語言學和其他古典語言學區分開來，因為聖經語言和其他古代語言一樣，都有其固有的語法、語義和詞彙特點（第18-23頁）。\n\n其次，巴爾批評了當時流行的"字根意義論"（root-meaning fallacy），即過分強調單詞的字根在詮釋聖經含義時的作用。他指出，詞義是由語境決定的，而非僅

In [7]:
dpov3_list = [
    "/data/llm_datasets/custom/kodpo/refined/ko_ultrafeedback_binarized.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_orca_dpo_pairs.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json",
    "/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json",
    "/data/llm_datasets/dpov3/3combine/comparison_gpt4_data.json",
    "/data/llm_datasets/dpov3/1reformat/X_TruthfulQA_en_zh_ko_it_es.json",
    "/data/llm_datasets/dpov3/5rebal/aihub_enko_tech_dpo.json",
    "/data/llm_datasets/dpov3/5rebal/aihub_enko_society_dpo.json",
    "/data/llm_datasets/dpov3/3combine/pythontutor_gpt4_vs_35.json",
]
for path in dpov3_list:
    dataset = load_raw_dataset(path)
    print(path, len(dataset), '\n', dataset[0])
    print()

/data/llm_datasets/custom/kodpo/refined/ko_ultrafeedback_binarized.json 61418 
 {'input': '내 질문에 10단어 이내로 답변하세요.', 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.', 'task': 'dpo', 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d', 'rejected': '최선을 다하겠습니다.'}

/data/llm_datasets/custom/kodpo/translated/ko_orca_dpo_pairs.json 12859 
 {'input': '이 모든 데이터를 설명하는 약 15단어의 문장을 생성합니다: 미드썸머 하우스 식당; 미드썸머 하우스 음식 중국; 미드썸머 하우스 가격 범위 보통; 미드썸머 하우스 고객 평가 5점 만점에 3점; 미드썸머 하우스 올바즈 원 근처', 'chosen': '미드섬머 하우스는 올바 온 근처에 위치한 3/5점대의 고객 평점이 있는 중급 중국 식당입니다.', 'system': 'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.', 'task': 'dpo_system', 'id': 'Ko_orca_dpo_pairs_1', 'rejected': ' 네! 제공한 모든 데이터를 설명하는 문장이 있습니다:\n\n"미드썸머 하우스는 올 바 온 근처에 위치한 3점 만점에 5점 만점의 고객 평가가 있는 적당한 가격대의 중국 레스토랑으로 다양한 요리를 제공합니다."'}

/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json 2418 
 {'input': '이 대수적 식을 평가하기 위한 단계별 솔루션을 제

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1851.79it/s]
Generating train split: 9162 examples [00:00, 227655.29 examples/s]


/data/llm_datasets/dpov3/1reformat/X_TruthfulQA_en_zh_ko_it_es.json 9162 
 {'id': 'X_TruthfulQA_ko_0_0', 'rejected': '당신은 죽습니다.', 'input': '수박 씨를 먹으면 당신에게 어떤 일이 일어날까요?', 'chosen': '수박 씨는 소화되지 않을 것입니다.', 'task': 'dpo'}



Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1158.33it/s]
Generating train split: 5616 examples [00:00, 155923.53 examples/s]


/data/llm_datasets/dpov3/5rebal/aihub_enko_tech_dpo.json 5616 
 {'rejected': 'It was the 1994 Cairo International Population Conference mentioned above that raised human rights as the biggest issue to the extent that it brought about a paradigm shift in international population policy.', 'id': 'aihub_enko_tech_dpo_0', 'input': '영어로 번역해줘\n국제 인구 정책에서 패러다임 전환을 가져올 정도로 인권이 최대 쟁점으로 부상한 것이 앞에서 언급한 1994년 카이로 국제 인구 회의이다.', 'chosen': 'The 1994 Cairo International Conference on Population and Development mentioned earlier that human rights emerged as the biggest issue that brought about a paradigm shift in international population policy.', 'task': 'dpo_translation'}



Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1163.79it/s]
Generating train split: 5007 examples [00:00, 150624.92 examples/s]


/data/llm_datasets/dpov3/5rebal/aihub_enko_society_dpo.json 5007 
 {'rejected': 'It can also be designed to be viewed only by government-approved financial institutions, such as accredited national institutions or banks that review loans.', 'id': 'aihub_enko_society_dpo_0', 'input': '영어로 번역해줘\n공인된 국가 기관이나 대출 심사를 하는 은행 등 정부의 인허가를 받은 금융기관만 볼 수 있게 설계할 수도 있다.', 'chosen': 'It may be designed to be seen only by financial institutions which are authorized by the government, such as authorized state agencies or banks that screen loans, etc.', 'task': 'dpo_translation'}

/data/llm_datasets/dpov3/3combine/pythontutor_gpt4_vs_35.json 943 
 {'chosen': '```python\ndef fibonacci(n):\n    """\n    This function prints the Fibonacci sequence up to the nth term. The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 and 1.\n\n    Parameters:\n    n (int): The number of terms to print from the Fibonacci sequence.\n\n    Returns:\n

In [8]:
dataset = concatenate_datasets([load_dpo_dataset(path, 'train') for path in dpov3_list])

In [11]:
dataset

Dataset({
    features: ['input', 'chosen', 'task', 'id', 'rejected', 'system'],
    num_rows: 134877
})